# Train your first 🐸 TTS model 💫

### 👋 Hello and welcome to Coqui (🐸) TTS

The goal of this notebook is to show you a **typical workflow** for **training** and **testing** a TTS model with 🐸.

Let's train a very small model on a very small amount of data so we can iterate quickly.

In this notebook, we will:

1. Download data and format it for 🐸 TTS.
2. Configure the training and testing runs.
3. Train a new model.
4. Test the model and display its performance.

So, let's jump right in!


In [ ]:
## Install Coqui TTS
! pip install -U pip
! pip install TTS


In [ ]:
!sudo apt-get install espeak

In [ ]:
!tts -h

## ✅ Data Preparation

### **First things first**: we need some data.

We're training a Text-to-Speech model, so we need some _text_ and we need some _speech_. Specificially, we want _transcribed speech_. The speech must be divided into audio clips and each clip needs transcription. More details about data requirements such as recording characteristics, background noise and vocabulary coverage can be found in the [🐸TTS documentation](https://tts.readthedocs.io/en/latest/formatting_your_dataset.html).

If you have a single audio file and you need to **split** it into clips. It is also important to use a lossless audio file format to prevent compression artifacts. We recommend using **wav** file format.

The data format we will be adopting for this tutorial is taken from the widely-used  **LJSpeech** dataset, where **waves** are collected under a folder:

<span style="color:purple;font-size:15px">
/wavs<br />
 &emsp;| - audio1.wav<br />
 &emsp;| - audio2.wav<br />
 &emsp;| - audio3.wav<br />
  ...<br />
</span>

and a **metadata.csv** file will have the audio file name in parallel to the transcript, delimited by `|`:

<span style="color:purple;font-size:15px">
# metadata.csv <br />
audio1|This is my sentence. <br />
audio2|This is maybe my sentence. <br />
audio3|This is certainly my sentence. <br />
audio4|Let this be your sentence. <br />
...
</span>

In the end, we should have the following **folder structure**:

<span style="color:purple;font-size:15px">
/MyTTSDataset <br />
&emsp;| <br />
&emsp;| -> metadata.csv<br />
&emsp;| -> /wavs<br />
&emsp;&emsp;| -> audio1.wav<br />
&emsp;&emsp;| -> audio2.wav<br />
&emsp;&emsp;| ...<br />
</span>

🐸TTS already provides tooling for the _LJSpeech_. if you use the same format, you can start training your models right away. <br />

After you collect and format your dataset, you need to check two things. Whether you need a **_formatter_** and a **_text_cleaner_**. <br /> The **_formatter_** loads the text file (created above) as a list and the **_text_cleaner_** performs a sequence of text normalization operations that converts the raw text into the spoken representation (e.g. converting numbers to text, acronyms, and symbols to the spoken format).

If you use a different dataset format then the LJSpeech or the other public datasets that 🐸TTS supports, then you need to write your own **_formatter_** and  **_text_cleaner_**.

## ⏳️ Loading your dataset
Load one of the dataset supported by 🐸TTS.

We will start by defining dataset config and setting LJSpeech as our target dataset and define its path.


In [4]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)


In [5]:
!wget https://huggingface.co/datasets/ntt123/viet-tts-dataset/resolve/main/viet-tts.tar.gz -O viet-tts.tar.gz
!mkdir -p dataset
!tar -C dataset -xzf viet-tts.tar.gz

--2023-07-20 15:28:37--  https://huggingface.co/datasets/ntt123/viet-tts-dataset/resolve/main/viet-tts.tar.gz
Resolving huggingface.co (huggingface.co)... 108.138.94.27, 108.138.94.45, 108.138.94.52, ...
Connecting to huggingface.co (huggingface.co)|108.138.94.27|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b6/a0/b6a06f111e29ec13cce1bd8ac23c03399a91a255436d45e3107e1166e1aae0eb/e00f71ccfe907cd74a9cb1bcb60d058ec8a9aa8c4c101d019dbeeefc588c5d0b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27viet-tts.tar.gz%3B+filename%3D%22viet-tts.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1690126118&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MDEyNjExOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iNi9hMC9iNmEwNmYxMTFlMjllYzEzY2NlMWJkOGFjMjNjMDMzOTlhOTFhMjU1NDM2ZDQ1ZTMxMDdlMTE2NmUxYWFlMGViL2UwMGY3MWNjZmU5MDdjZDc0YTljYjFiY2I2MGQw

In [ ]:
# Download and extract LJSpeech dataset.

!wget -O $output_path/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xf $output_path/LJSpeech-1.1.tar.bz2 -C $output_path

In [16]:
import csv

file_path = '/content/dataset/meta_data.tsv'  # Replace with the actual path to your text file

# Method 1: Using the 'open' function
count = 0
data = []
with open(file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        count += 1
        # print(line.strip().split("\t"))
        name_file, text_file = line.strip().split("\t")
        format_name = name_file.split("/")[1].split(".")[0]
        data.append([format_name,text_file,text_file])
        if count == 17000:
          break
file.close()
print(len(data))
output_file = '/content/dataset/metadata.csv'

# Write the data to the CSV file with "|" as the delimiter
with open(output_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter='|')  # Set the delimiter to "|"
    csv_writer.writerows(data)

print("Data has been written to the CSV file.")
csvfile.close()


UnicodeDecodeError: ignored

In [ ]:
from IPython.display import clear_output
file_path =  '/content/tts_train_dir/LJSpeech-1.1/metadata.csv'  # Replace with the actual path to your text file

# Method 1: Using the 'open' function
count = 0
with open(file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        count += 1
        # print(line.strip())
        raw = line.strip().split("|")
        for i in raw :
          print(i)
        if count == 50 :
          break

# clear_output()

In [8]:
# dataset_config = BaseDatasetConfig(
#     formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "LJSpeech-1.1/")
# )

dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path="/content/dataset"
)

## ✅ Train a new model

Let's kick off a training run 🚀🚀🚀.

Deciding on the model architecture you'd want to use is based on your needs and available resources. Each model architecture has it's pros and cons that define the run-time efficiency and the voice quality.
We have many recipes under `TTS/recipes/` that provide a good starting point. For this tutorial, we will be using `GlowTTS`.

We will begin by initializing the model training configuration.

In [9]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=3,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

Next we will initialize the audio processor which is used for feature extraction and audio I/O.

In [10]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)
# Modify sample rate if for a custom audio dataset:
# ap.sample_rate = 22050


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


Next we will initialize the tokenizer which is used to convert text to sequences of token IDs.  If characters are not defined in the config, default characters are passed to the config.

In [11]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

Next we will load data samples. Each sample is a list of ```[text, audio_file_path, speaker_name]```. You can define your custom sample loader returning the list of samples.

In [12]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 21000 files in /content/dataset


Now we're ready to initialize the model.

Models take a config object and a speaker manager as input. Config defines the details of the model like the number of layers, the size of the embedding, etc. Speaker manager is used by multi-speaker models.

In [13]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training, distributed training, etc.

In [14]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=tts_train_dir/run-July-20-2023_03+33PM-0000000

 > Model has 28610257 parameters


### AND... 3,2,1... START TRAINING 🚀🚀🚀

In [15]:
trainer.fit()


 > EPOCH: 0/3
 --> tts_train_dir/run-July-20-2023_03+33PM-0000000


[*] Pre-computing phonemes...


 21%|██        | 4289/20790 [03:07<10:40, 25.77it/s]

ˈɑːŋ kˈoʊ bˈiː ˈaɪ lˌɛɾɚwˈʌnˈiːbˈiːˈɛf tˈiː kˈɑːŋ? nˈoʊ vˈiː lˌɛɾɚwˈʌnˈiːdˈiːbˈiː ˈaɪ kʃˈuːæn ˌɑːɹlˌɛɾɚwˌʌnˈiːˌiːsˈɛvən ˈɛnhˈaʊ vˈæoʊ ˌɛmlˌɛɾɚwˌʌnˈiːdˌiːnˌaɪntˈiː ˈoʊʔn̩ θˈʌɛː bˈiː jˈuː lˌɛɾɚwˈʌnˈiːdˈiːθɹˈiː ˈɛn dʒˈiː! sˌiːˌeɪtʃlˌɛɾɚwˈʌnˌiːbˌiːˌɛftˈiː tˌiːˌeɪtʃlˌɛɾɚwˈʌnˌiːˌeɪdˌiːtˈiː sˌiːˌeɪtʃlˌɛɾɚwˈʌnˌiːˌiːnˈaɪn kˈɑːŋ ˌɛndʒˌiːlˌɛɾɚwˈʌnˌiːdˌiːdˈiː ˌɑːɹlˌɛɾɚwˌʌnˈiːbˌiːwˌʌnˌɛndʒˈiː...
 [!] Character '̩' not found in the vocabulary. Discarding it.


100%|██████████| 20790/20790 [15:16<00:00, 22.70it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > TRAINING (2023-07-20 15:49:29) 


...


   --> TIME: 2023-07-20 15:49:43 -- STEP: 0/650 -- GLOBAL_STEP: 0
     | > current_lr: 2.5e-07 
     | > step_time: 11.7519  (11.751869440078735)
     | > loader_time: 3.1018  (3.101792097091675)

 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.

   --> TIME: 2023-07-20 15:49:58 -- STEP: 25/650 -- GLOBAL_STEP: 25
     | > loss: 1.112144947052002  (1.1652427275975545

Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1809, in fit
    self._fit()
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1761, in _fit
    self.train_epoch()
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1490, in train_epoch
    outputs, _ = self.train_step(batch, batch_num_steps, cur_step, loader_start_time)
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1348, in train_step
    outputs, loss_dict_new, step_time = self.optimize(
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1212, in optimize
    outputs, loss_dict = self._compute_loss(
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1143, in _compute_loss
    outputs, loss_dict = self._model_train_step(batch, model, criterion)
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1102, in _model_train_step
    return mo

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1809, in fit
    self._fit()
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1761, in _fit
    self.train_epoch()
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1490, in train_epoch
    outputs, _ = self.train_step(batch, batch_num_steps, cur_step, loader_start_time)
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1348, in train_step
    outputs, loss_dict_new, step_time = self.optimize(
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1212, in optimize
    outputs, loss_dict = self._compute_loss(
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1143, in _compute_loss
    outputs, loss_dict = self._model_train_step(batch, model, criterion)
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1102, in _model_train_step
    return mo

TypeError: ignored

#### 🚀 Run the Tensorboard. 🚀
On the notebook and Tensorboard, you can monitor the progress of your model. Also Tensorboard provides certain figures and sample outputs.

In [ ]:
# !pip install tensorboard
# !tensorboard --host localhost --logdir=tts_train_dir

## ✅ Test the model

We made it! 🙌

Let's kick off the testing run, which displays performance metrics.

We're committing the cardinal sin of ML 😈 (aka - testing on our training data) so you don't want to deploy this model into production. In this notebook we're focusing on the workflow itself, so it's forgivable 😇

You can see from the test output that our tiny model has overfit to the data, and basically memorized this one sentence.

When you start training your own models, make sure your testing data doesn't include your training data 😅

Let's get the latest saved checkpoint.

In [ ]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
!tts --text "I'M a frieezze" \
      --model_path /content/tts_train_dir/run-July-20-2023_09+40AM-0000000/best_model_406.pth \
      --config_path /content/tts_train_dir/run-July-20-2023_09+40AM-0000000/config.json \
      --out_path out.wav

NotImplementedError: ignored

## 📣 Listen to the synthesized wave 📣

In [ ]:
import IPython
IPython.display.Audio("out.wav")

## 🎉 Congratulations! 🎉 You now have trained your first TTS model!
Follow up with the next tutorials to learn more advanced material.

In [ ]:
from google.colab import files
files.download('/content/tts_train_dir/run-July-20-2023_12+01PM-0000000/config.json')
files.download('/content/dataset/metadata.csv')